# Torch Tensors



This notebook explores the tensor datatype in detail.

In [1]:
import torch
import numpy as np

In [2]:
t = torch.Tensor()

## Attributes of a tensor

A tensor object has by default the following three attributes:

A Datatype - Specifying the type of the class which an object belongs to.
```python
print(t.dtype)
```
A Device - Whether this object lives on the CPU or the GPU.
```python
print(t.device)
```
Layout - How the data is stored internally. Not necessarily important to know. Default is good enough.
```python
print(t.layout)
```

In [4]:
print(t.dtype)
print(t.device)
print(t.layout)

torch.float32
cpu
torch.strided


Torch tensors have a datatype and the operations in tensors require the datatype of two tensors to be of the same type. It is not possible to operate on tensors with different datatypes.

A torch tensor that lives in a gpu vs a torch tensor that lives in cpu has two different datatypes. And it's not possible to operate on them in this state. Either use the `.device('cuda')` to get the cpu tensor on the gpu or vice versa in order to make it happen.

## Creating tensor with existing data.

There are several ways of creating a tensor with existing data like numpy arrays. They are as follows:

```python
torch.Tensor(data)
torch.tensor(data)
torch.as_tensor(data)
torch.from_numpy(data)
```
data is a numpy array which is already defined in memory.

**Note:** The first one is different because it's made using the constructor in the Tensor class as if you provide it an integer data or whatever data, it'll interpret it as float data whereas others will more or less be representative dtype of the source data.

In [12]:
data = np.array([[1, 2, 3], [4, 5, 6]])

In [13]:
torch.Tensor(data)

tensor([[1., 2., 3.],
        [4., 5., 6.]])

In [14]:
torch.tensor(data)

tensor([[1, 2, 3],
        [4, 5, 6]], dtype=torch.int32)

In [15]:
torch.as_tensor(data)

tensor([[1, 2, 3],
        [4, 5, 6]], dtype=torch.int32)

In [16]:
torch.from_numpy(data)

tensor([[1, 2, 3],
        [4, 5, 6]], dtype=torch.int32)

## Create tensor with predefined functions

Sometimes, we need to get an identity matrix or a matrix with all zeros or all ones etc. We can use the following functions for the same.

```python
torch.eye(n)
torch.zeros(m, n)    # For a tensor of all zeros
torch.ones(m, n)     # For a tensor of all ones
torch.rand(m, n)     # For a tensor of all random numbers
```

In [17]:
torch.eye(3)

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

In [18]:
torch.zeros(1, 2)

tensor([[0., 0.]])

In [19]:
torch.ones(3, 2)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

In [20]:
torch.rand(2,3)

tensor([[0.1446, 0.5538, 0.0546],
        [0.0184, 0.3180, 0.2586]])

## Memory Sharing and Copying

The two methods `torch.tensor` and `torch.Tensor` work copy the information from a numpy array into a tensor and a separate memory space is made available to them. OTOH, the methods `torch.as_tensor` and `torch.from_numpy` only create references which causes changes to reflect in the destination if they've happened in the source. i.e. If you change the numpy array, then the tensors created from that array will be modified if they're created using `as_tensor` or `from_numpy` methods. 

In [26]:
data = np.ones((3,3))

In [27]:
print(data)

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


In [28]:
t1 = torch.Tensor(data)
t2 = torch.tensor(data)
t3 = torch.as_tensor(data)
t4 = torch.from_numpy(data)

In [31]:
data[0][0] = 0

In [32]:
print(t1)
print(t2)
print(t3)
print(t4)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[0., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[0., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)


## Flatten, Reshape, Squeeze, Concatenate

To check the size of a tensor you can use
```python
t.size()
t.shape
```

To check the rank of a tensor, you can use 
```python
len(t.size())
```

To find out the number of elements contained in a tensor, you can use 
```python
t.numel()
```
This operation is particularly useful when you want to check the suitability of a new dimension for your tensor. We always want the number of elements of the source and destination tensor to be equal in order to ensure that the reshaping operation happens without an error.

To reshape the tensor to a new dimension you can use
```python
t.reshape(m,n,p)
```
With the caveat that source and destination should have same numel.

**Squeezing and unsqueezing** operations allow us to expand or contract the rank of a tensor. Squeezing a tensor removes the dimensions or axes that have a length of one. Unsqueezing a tensor adds a dimension with a length of 1. 

```python
t.reshape([1, t.numel()]).squeeze()
```

A **Flatten** operation takes in a tensor and converts it to a 1-D tensor of number of elements in the tensor. Flattening a tensor means to remove all of it's dimensions except for 1.

```python
def flatten(t):
    t = t.reshape(1, -1)
    t = t.squeeze()
    return t
```

We can concatenate one tensor alongside another either rowwise or columnwise. This can be handy when you want to collate two tensors for some reason. We can do it as follows:
```python
t1 = torch.tensor(data1)
t2 = torch.tensor(data2)

# Rowwise concatenation
tcombined = torch.cat((t1, t2), dim = 0)

# Columnwise concatenation
tcombined = torch.cat((t1, t2), dim = 1)
```

A **Stacking** operation takes in a lot of tensors and adds a dimension to create a batch of tensors. eg. If we have 1000 images, we can make batches of size 64 which involve stacking 64 images together into one batch and that would be represented along an axis of the tensor. Here's how this could be achieved.

```python
# Stack 3 tensors together
tcomb = torch.stack((t1, t2, t3))
```

In [37]:
# Flattening an image along color-channel component
t1 = torch.ones((3,3))
t2 = torch.ones((3,3)) * 2
t3 = torch.ones((3,3)) * 3

batch = torch.stack((t1, t2, t3))
batch = batch.reshape(3, 1, 3, 3)
print(batch)

tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[2., 2., 2.],
          [2., 2., 2.],
          [2., 2., 2.]]],


        [[[3., 3., 3.],
          [3., 3., 3.],
          [3., 3., 3.]]]])


In [38]:
batch.size()

torch.Size([3, 1, 3, 3])

In [41]:
# We need to flatten this across the color channel and across the height and the width
# It's easily possible by a built-in function but let's try something else

batch_manual = batch.reshape(3, batch.size()[1] * batch.size()[2] * batch.size()[3])
print(batch_manual)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3.]])


In [42]:
# Using a built in function
batch_func = batch.flatten(start_dim = 1)
print(batch_func)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3.]])


## Elementwise tensor operations

Two elements of a tensor are said to be corresponding if they both occupy the same position in their respective tensors. This restricts these operands to both be of the same shape. 

Elementwise operators are the ones which operate on corresponding elements in between two tensor operands. They are also called pointwise or componentwise operations.

However, this get's interesting when we have scalar values. When we operate a tensor with a scalar, it works (Well it shouldn't since the scalar is a rank 0 tensor and other tensors have non-zero rank so their shapes mismatch).

What happens under the hood is that the scalar value is broadcasted to be the same size/shape as the tensor on which it is operated and then the elementwise operation is carried out.

**In general, given two tensors, if the low rank tensor could be broadcasted to a high rank tensor, then the elementwise operation will happen no matter what, otherwise they won't and you'll get an error**.

Apart from the general arithmetic operations there are some other operations which you should be aware of and some of them are as follows. 

```python
# To check if elements of a tensor are greater than or equal to a number
t.ge(0)

# Less than or equal to a number
t.le(0)

# Greater than a number
t.gt(0)

# Less than a number
t.lt(0)

# Equal to a number
t.eq(0)

# Unary operators
# Square root
t.sqrt()

# Modulus function
t.abs()

# Negation
t.neg()

# Chained operations
t.sqrt().abs()
```

In [43]:
t = torch.tensor([[1,2],[3,4]], dtype = torch.float64)

In [44]:
t 

tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)

In [45]:
t + 2

tensor([[3., 4.],
        [5., 6.]], dtype=torch.float64)

In [47]:
t + torch.tensor(np.broadcast_to(2, t.shape), dtype = torch.float64)

tensor([[3., 4.],
        [5., 6.]], dtype=torch.float64)

In [48]:
t2 = torch.tensor([1,2,3], dtype = torch.float64)

In [51]:
t.shape

torch.Size([2, 2])

In [52]:
t2.shape

torch.Size([3])

In [55]:
t + t2

RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1

## Tensor Reduction operations

They are the operations which reduce the number of elements of a tensor. There are a lot of useful tensor reduction operations which we will use commonly when dealing with tensors. Some of them are as follows:

```python
# Input is a nd tensor output is a 1D tensor with a single scalar value

# Sum all the elements of a tensor 
t.sum()

# Find the mean & standard deviation of all elements in the tensor
t.mean()
t.std()

# Find the product of all the elements in the tensor
t.prod()
```

We can reduce the tensor across a dimension in order to avoid getting one single scalar value as outputs. The meaning changes. For eg. if you mention dim = 0 in a mean operation on a 2D tensor, it will give a tensor of 1D values which will correspond to the means of all the columns in the tensor that you passed.

```python
t.sum(dim = 0)  # For a 3 row matrix, t[0] + t[1] + t[2]
t.sum(dim = 1)  # For a 3 row matrix, [t[0].sum, t[1].sum, t[2].sum()]
```

**argmax** is an important reduction operation which returns the index of the element which has the highest value in the tensor (Or a tensor depending along a user specific dimension if you provide one).

```python
t.argmax(dim = 1)
```

If you want to access a scalar element as the result of an operation, you could do the following
```python
t.sum().item()
```
Note that the output of `t.sum()` will be returning a tensor containing a scalar value but it might not be something that you want. You just want the scalar value. That time you can use `.item` to get the scalar value.

Also, if you operate along a row or column and get a 1-D tensor but want to operate on it in python, you could convert it to a list or into a numpy array and do the necessary processing.
```python
t.sum(axis = 1).tolist()
t.sum(axis = 1).numpy()
```
